# Is Success Luck or Hard Work?

This short script is basically me checking out how not just looking at luck, but also looking at how skill is distributed, affects the results of the experiment proposed at [this time stamp](https://youtu.be/3LopI4YeC4I?t=212) of the youtube video.

In [1]:
import numpy as np
from functools import partial
from numpy.random import uniform, normal

number_of_candidates = 18_300
number_of_graduates = 11
skill_weight = 0.95
luck_weight = 0.05
number_of_simulations = 1000

In [2]:
def how_many_wouldve_graduated_anyway(random_function):
    simulations_results = np.empty(number_of_simulations)
    for simulation_number in range(number_of_simulations):
        # Generate random test results
        test_results = random_function(size=(number_of_candidates, 3))
        test_results[:, 0] *= skill_weight  # skill
        test_results[:, 1] *= luck_weight  # luck
        test_results[:, 2] = test_results[:, 0] + test_results[:, 1]  # total score
        
        # Truncate random numbers between 0 and 100
        test_results[test_results > 100] = 100
        test_results[test_results < 0] = 0
        
        # Select top 11 candidates who will graduate, plus the top non-graduate to act
        # as 'graduatability' threshold
        top_k = number_of_graduates + 1
        top_candidates_indexes = np.argpartition(test_results[:, 2], -top_k)[-top_k:]
        top_candidates = test_results[top_candidates_indexes]

        # Get graduation threshold score
        top_non_graduate_index = np.argmin(top_candidates[:, 2])
        graduation_threshold_score = top_candidates[top_non_graduate_index][2]

        # Get number of graduates who would've graduated with their luck set to 0
        graduates = np.delete(top_candidates, top_non_graduate_index, axis=0)
        number_who_wouldve_graduated_anyway = np.sum(graduates[:, 0] > graduation_threshold_score)
        simulations_results[simulation_number] = number_who_wouldve_graduated_anyway
        
    return np.mean(simulations_results)

In [3]:
# Here I get a similar result to the one mentioned in the video.
how_many_wouldve_graduated_anyway(
    partial(normal, loc=50, scale=5)
)

1.503

In [4]:
# But I can change it if I increase the spread of the data.
# So if I didn't screw anything up, it would seem that luck's
# influence is actually dependent on how rare skill is.
how_many_wouldve_graduated_anyway(
    partial(normal, loc=50, scale=10)
)

4.185

In [5]:
# As expected, luck is necessary when skill is uniformly distributed.
how_many_wouldve_graduated_anyway(
    partial(uniform, low=0, high=100)
)

0.0

In [6]:
# PD: I guess I could've mantained the luck distribution fixed among tests,
# but I got lazy. I think I would've gotten similar results though.

# Veritasium rocks!